## July 22 2021

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
start_time = time.time()
df = pd.read_feather('D:/internship-telkom/data-science/slow-connection/CW02_dataset.ftr')
print('Time elapsed: {}'.format((time.time() - start_time)))

AttributeError: module 'pandas' has no attribute 'read_feather'

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_feather('D:/internship-telkom/data-science/slow-connection/CW02_dataset.ftr')

def make_df_bin(bin_1h, sample):
    cols = ['user_id', 'date_time', 'reportdate', 'bin_1h', 'int_latency_score', 'ext_latency_score', 'fd_latency_score',
            'int_ploss_score', 'ext_ploss_score', 'ses_setup_score', 'ser_acces_score']

    report1 = df[cols][(df['bin_1h'] >= bin_1h) & (df['bin_1h'] < 0)].drop_duplicates().sample(sample)
    report1.drop('user_id', axis=1, inplace=True)
    report1['report_0_1'] = 1
    
    report0 = df[cols][(df['bin_1h'] < bin_1h) | (df['bin_1h'] > 0)].drop_duplicates().sample(sample)
    report0.drop('user_id', axis=1, inplace=True)
    report0['report_0_1'] = 0
    
    df_report = pd.concat([report1, report0])
    return df_report

        
def make_df_score(score_name, bin_1h=-48, sample=100000):
    try:
        cols = ['name', 'score1', 'score2', 'score3', 'score4', 'scoremin1', 'report']
        score_lst = [1, 2, 3, 4, -1]

        df_report = make_df_bin(bin_1h, sample)
        df_score = pd.DataFrame(columns=cols)
        df_score['report'] = [0, 1]
        for n1, score in zip(range(1, 6), score_lst):
            for n2 in range(0, 2):
                df_score[cols[n1]][df_score['report'] == n2] = df_report[(df_report['report_0_1'] == n2) &
                                                                         (df_report[score_name] == score)].shape[0]                                                                  
        df_score['name'] = score_name
        return df_score
    except:
        print('Error! Please specify less smaller sample size.')


cols = [col for col in df.columns if 'score' in col]

df_conc = pd.concat([make_df_score(cols[0]), make_df_score(cols[1]), make_df_score(cols[2]), make_df_score(cols[3]),
                     make_df_score(cols[4]), make_df_score(cols[5]), make_df_score(cols[6])])
df_conc.reset_index(drop=True, inplace=True)
df_conc[['score1', 'score2', 'score3', 'score4', 'scoremin1']] = round(df_conc[['score1', 'score2', 'score3', 'score4', 'scoremin1']].astype(int) / 1000, 2)
print(df_conc)

                 name  score1  score2  score3  score4  scoremin1  report
0   int_latency_score   81.29    5.09    2.03    3.87       7.72       0
1   int_latency_score   75.65    5.44    2.37    4.49      12.04       1
2   ext_latency_score   89.06    2.33    0.29    0.29       8.03       0
3   ext_latency_score   84.32    2.03    0.24    0.29      13.13       1
4    fd_latency_score   43.60   26.69    9.37   14.73       5.61       0
5    fd_latency_score   39.64   24.81    9.01   15.90      10.65       1
6     int_ploss_score   71.06   25.72    1.47    1.56       0.18       0
7     int_ploss_score   66.44   26.68    1.96    2.27       2.65       1
8     ext_ploss_score   86.84   10.25    0.71    0.63       1.58       0
9     ext_ploss_score   82.87   11.27    0.97    2.59       2.30       1
10    ses_setup_score   89.18    8.07    1.65    0.90       0.20       0
11    ses_setup_score   88.57    8.09    1.95    1.08       0.32       1
12    ser_acces_score   19.61   50.23   21.33    8.

In [ ]:
df = pd.read_feather('D:/big-datasets/internship-telkom/Archive 3/')